<a href="https://colab.research.google.com/github/vanyagoncharov/CourseMLResourse/blob/main/dz_23_s3e6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозировании цен на жилье в Париже

## Импорт библиотек

In [50]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

## Функции

## Загрузка данных

In [28]:
# Загружаем тестовый и тренировочный наборы данных
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/playground-series-s3e6/test.csv')
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/playground-series-s3e6/train.csv')

In [29]:
# Удаление ненужных столбцов, которые могут не способствовать прогнозированию модели.
data_test = data_test.drop(['id'], axis=1)
data_train = data_train.drop(['id'], axis=1)

In [30]:
data_test

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
0,47580,89,0,1,8,54830,5,3,1995,0,0,6885,8181,241,0,8
1,62083,38,0,0,87,8576,10,3,1994,1,1,4601,9237,393,1,4
2,90499,75,1,1,37,62454,9,6,1997,0,1,7454,2680,305,0,2
3,16354,47,1,1,9,9262,6,5,2019,1,1,705,5097,122,1,5
4,67510,8,0,0,55,24112,3,7,2014,1,1,3715,7979,401,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,86184,74,1,1,73,14009,2,10,1993,1,0,3714,3894,665,1,1
15150,45885,41,1,1,64,84208,4,5,2008,0,0,783,7628,976,0,0
15151,73554,51,0,0,41,49263,10,5,2004,0,1,8207,1777,775,0,10
15152,77604,51,0,1,33,67056,4,3,2019,0,0,9578,9706,135,0,2


In [31]:
data_train

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,55825,84,1,0,70,12031,3,10,2000,0,0,4477,786,345,0,0,5594137.1
22726,65870,88,1,0,49,23197,9,9,2015,0,1,4811,2454,755,0,7,6594705.0
22727,93192,42,1,0,39,8539,10,5,2014,1,0,5595,4072,789,0,0,9321511.4
22728,65797,86,1,0,89,23197,2,10,2000,1,0,5358,2513,411,0,0,6584708.2


## Разведочный анализ данных

### Общий анализ данных

In [36]:
pd.value_counts(data_train.dtypes)

int64      16
float64     1
dtype: int64

In [37]:
pd.value_counts(data_test.dtypes)

int64    16
dtype: int64

### Сколько строк в датафрейме, сколько столбцов

In [33]:
# Вывод информации о датасете
print(f'Информация о датасете: \n {data_train.shape[0]}: строк\n {data_train.shape[1]}: столбцов\n')

Информация о датасете: 
 22730: строк
 17: столбцов



### Сколько места занимает датафрейм в оперативной памяти

In [34]:
print(f'{np.round(data_train.memory_usage(index=True).sum() / 1024, 4)} КБ: занимаемый объем памяти')

3018.9531 КБ: занимаемый объем памяти


### Для каждой интервальной переменной подсчитать следующее - мин, медиана, среднее, макс и персентили 25, 75

In [35]:
data_train.describe()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
count,2.273000e+04,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,22730.000000,2.273000e+04
mean,4.658622e+04,48.241091,0.475891,0.452750,47.305983,50013.795996,5.585042,5.620766,2008.005059,0.467972,0.460009,5283.646634,5067.981698,530.469644,0.461681,5.153058,4.634457e+06
std,4.952124e+04,28.226428,0.499429,0.497773,47.777207,30006.637729,2.739533,2.713026,118.826777,0.498984,0.498409,3047.084412,3097.347939,274.840604,0.498540,3.055246,2.925163e+06
min,8.900000e+01,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,1.000000,1990.000000,0.000000,0.000000,4.000000,1.000000,4.000000,0.000000,0.000000,1.031350e+04
25%,2.039275e+04,25.000000,0.000000,0.000000,25.000000,22936.000000,3.000000,3.000000,2000.000000,0.000000,0.000000,2977.250000,2599.000000,297.000000,0.000000,3.000000,2.041739e+06
50%,4.448400e+04,47.000000,0.000000,0.000000,45.000000,50414.000000,6.000000,6.000000,2006.000000,0.000000,0.000000,5359.000000,4977.000000,515.000000,0.000000,5.000000,4.450823e+06
75%,7.154700e+04,75.000000,1.000000,1.000000,69.000000,76291.000000,8.000000,8.000000,2014.000000,1.000000,1.000000,7746.000000,7652.000000,767.000000,1.000000,8.000000,7.159920e+06
max,6.071330e+06,100.000000,1.000000,1.000000,6000.000000,491100.000000,10.000000,10.000000,10000.000000,1.000000,1.000000,91992.000000,96381.000000,9017.000000,1.000000,10.000000,1.000428e+07


### Для каждой категориальной переменной рассчитать моду и сколько раз мода встречается в данных

In [43]:
data_train.dtypes

squareMeters           int64
numberOfRooms          int64
hasYard                int64
hasPool                int64
floors                 int64
cityCode               int64
cityPartRange          int64
numPrevOwners          int64
made                   int64
isNewBuilt             int64
hasStormProtector      int64
basement               int64
attic                  int64
garage                 int64
hasStorageRoom         int64
hasGuestRoom           int64
price                float64
dtype: object

In [44]:
data_test.dtypes

squareMeters         int64
numberOfRooms        int64
hasYard              int64
hasPool              int64
floors               int64
cityCode             int64
cityPartRange        int64
numPrevOwners        int64
made                 int64
isNewBuilt           int64
hasStormProtector    int64
basement             int64
attic                int64
garage               int64
hasStorageRoom       int64
hasGuestRoom         int64
dtype: object

Исходя из анализа было видно, что категориальные переменные отсутствуют

## Подготовка датасета к построению моделей ML

### Провести анализ и обработку пропусков (либо заменить, либо удалить)

In [40]:
data_test.isnull().sum()

squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityCode             0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
dtype: int64

In [41]:
data_train.isnull().sum()

squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityCode             0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
price                0
dtype: int64

Пропуски в датасете отсутствуют

### Провести анализ и обработку выбросов (либо заменить, либо удалить)


https://www.kaggle.com/code/polaninethajiram/house-price-regression

In [ ]:
# Получение интервальных колонок из датасета
interval_columns = data_train.select_dtypes(include=np.number)
interval_columns_names = list(interval_columns)
print(interval_columns_names)
plt.figure(figsize=(16,20))

for i,col in enumerate(interval_columns_names):
    plt.subplot(3,3,i + 1)
    sns.boxplot(y=col, data=interval_columns)
    #plt.ylabel('')

plt.tight_layout()
plt.show()

### Провести анализ и обработку категориальных переменных (сколько таких переменных, закодируйте категориальные переменные одним из методов ( one hot encoding, mean target, frequence encoding)

Исходя из анализа выше было видно, что категориальные переменные отсутствуют

### Разделить датасет на трейн и тест

## Обучить любые 2 алгоритма:

### SVM

### Decision tree

## Оценить качество алгоритмов, выбрать самый оптимальный алгоритм

Затем сравните их производительность, используя метрики, такие как MAE, MSE, R^2, на тестовых данных. Обоснуйте свой выбор лучшей модели, учитывая точность
прогнозирования и обобщающую способность на новых данных